## Install Required libraries


In [ ]:
!pip install -r .\requirements.txt

Running the Code

In [ ]:
import os
# from dotenv import load_dotenv    #If using .env file for using API based LLMs

In [ ]:
MODEL = "phi"

In [ ]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings

model = OllamaLLM(model=MODEL)

embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "say i don't know"

context: {context}

question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"data\Sample.pdf")

pages = loader.load_and_split()

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    pages,
    embedding = embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# chain.invoke({"question":"Why do ionic compounds have high melting points?"})

In [ ]:
def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

In [ ]:
while True:
    q = input('Your question: ')
    if q.lower() in 'exit quit bye':
        print('Bye bye!')
        break
    result = ask_question(q, chain)
    print(result)
    print('-' * 100)